In [137]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [138]:
file=os.path.join(dirname, filename)
data=pd.read_csv(file,index_col=0)
data.head()
# data.shape
# data.describe()
# data.info()

In [139]:
data['diagnosis'].value_counts()

In [140]:
data.drop("Unnamed: 32",axis=1,inplace=True)


In [141]:
import matplotlib.pyplot as plt
import seaborn as sns

In [142]:
# find col with missing values 
for column in data.columns:
    mv=data[column].isna().sum()
    if(mv>0):
        print("{} has {} missing values".format(column,mv))

#data.radius_mean.isna().sum()

In [143]:
#find outliers for numeric columns
numeric_col=data.select_dtypes(include=['int64','float64']).columns
# #cat_col=data.select_dtypes(include=['object', 'boolean']).columns
# i=0
# for column in numeric_col:
#     i=i+1
#     plt.subplot(2,3,i)
#     plt.figure(figsize=(12,10))
#     #round(len(numeric_col)/3)
#     data[column].plot(kind="box",title=column)
for column in numeric_col:
    data[column].plot(kind="box",title=column)
    plt.show()


In [144]:
for column in numeric_col:
    l,u=np.percentile(data[column],[10,90])
    data[column]=np.clip(data[column],l,u)
    
#data["radius_mean"].head()

In [145]:
# check if outliers are removed
for column in numeric_col:
    #data[column].plot(kind="box",title=column)
    plt.title(column)
    plt.boxplot(data[column])
    plt.show()


In [146]:
#check for multi collinearity - visualization

plt.figure(figsize=(15,15))
sns.heatmap(data.corr())

In [147]:
data.head()
X=data.iloc[:,1:]
y=data.iloc[:,0]


In [148]:
#variane inflation factor 
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif_dict={ X.columns[i]:variance_inflation_factor(X.values,i) for i in np.arange(len(X.columns))}
# dict_val=list(vif_dict.values())
# dict_key=list(vif_dict.keys())
# dict_key[dict_val.index(max(dict_val))]
vif_dict

In [149]:
data.head()
from sklearn.preprocessing import LabelEncoder
data["diagnosis"]=LabelEncoder().fit_transform(data["diagnosis"])
# y.name
# y_numeric=pd.Series(z,name=y.name)
data["diagnosis"]

In [150]:
data.corr()

In [151]:
data_mod=data.copy()
data_mod.drop(["radius_mean","area_mean","radius_worst","area_worst","radius_se","perimeter_se","compactness_mean","concavity_mean","compactness_se","concavity_se","compactness_worst","concavity_worst","perimeter_mean","concave points_worst","fractal_dimension_se","fractal_dimension_mean","smoothness_mean","texture_mean","symmetry_mean","fractal_dimension_worst"],inplace=True,axis=1)

In [152]:
plt.figure(figsize=(10,6))
sns.heatmap(data_mod.corr())

In [153]:
X_sel=data_mod.iloc[:,1:]
vif_dict={ X_sel.columns[i]:variance_inflation_factor(X_sel.values,i) for i in np.arange(len(X_sel.columns))}
# dict_val=list(vif_dict.values())
# dict_key=list(vif_dict.keys())
# dict_key[dict_val.index(max(dict_val))]
vif_dict

In [154]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_sel,data_mod.iloc[:,0],test_size=0.3,random_state=0)
y_train.shape

In [155]:
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
X_train=ss.fit_transform(X_train)
X_test=ss.transform(X_test)

In [156]:
from sklearn.linear_model import LogisticRegression
classifier=LogisticRegression()
classifier.fit(X_train,y_train)
classifier.coef_

In [157]:
from sklearn import metrics

In [158]:
y_predit=classifier.predict(X_test)
classifier.score(X_train,y_train)
classifier.score(X_test,y_test)
y_test_prob_1=classifier.predict_proba(X_test)[:,1]
print(metrics.classification_report(y_test,y_predit))
print(metrics.confusion_matrix(y_test,y_predit))


In [159]:
fpr,tpr,thresholds=metrics.roc_curve(y_test,y_test_prob_1)
j=tpr-fpr
idx=np.argmax(j)
best_threshold=thresholds[idx]
best_threshold

In [160]:
y_predict_1_best_thr=np.where(y_test_prob_1>= best_threshold,1,0)
print("accuracy of logistic reg with default threshold(0.5) is {}".format(metrics.accuracy_score(y_test,y_predit)))
print("accuracy of logistic reg with best threshold(0.3678) is {}".format(metrics.accuracy_score(y_test,y_predict_1_best_thr)))
print(metrics.confusion_matrix(y_test,y_predict_1_best_thr))


In [175]:
p,r,thresholds=metrics.precision_recall_curve(y_test,y_test_prob_1)
f_score=(2*p*r)/(p+r)
idx=np.argmax(f_score)
best_threshold_pr=thresholds[idx]
print("best threshold by using f1 score %.010f"%best_threshold_pr)

# As false negative should be reduced, recall should be used
idx_r=np.argmax(r)
best_threshold_r=thresholds[idx]
print("best threshold by using recall score {}".format(best_threshold_r))

# using high Beta score as FN shoukd be reduced
beta=2
f_beta=((1+beta*beta)/(beta*beta))*((p*r)/(p+r))
idx=np.argmax(f_beta)
best_threshold_fbeta=thresholds[idx]
print("best threshold by using fbeta is {}".format(best_threshold_fbeta))



In [183]:
#hyper parameteer tuning with gridSerachCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
my_cv=RepeatedStratifiedKFold(n_splits=10,n_repeats=3,random_state=1)
log_model=LogisticRegression()
solver_list=['lbfgs','liblinear','sag','saga','newton-cg']
penalty_list=['l1','l2','none']
c_list=[100,10,1,0.1,0.001]
grid=dict(solver=solver_list,penalty=penalty_list,C=c_list)
grid_search=GridSearchCV(estimator=log_model,param_grid=grid,cv=my_cv,scoring='accuracy',error_score=0)
gridSearch_result=grid_search.fit(X_train,y_train)


In [198]:
print("gridSearch_result best score is {}".format(gridSearch_result.best_score_))
print("gridSearch_result best params are {}".format(gridSearch_result.best_params_))
print("gridSearch_result best_estimator_ are {}".format(gridSearch_result.best_estimator_))

In [197]:
various_params=gridSearch_result.cv_results_['params']
mean_score=gridSearch_result.cv_results_['mean_test_score']
for score,param in zip(mean_score,various_params):
    print("{} has score of {}".format(param,score))

In [210]:
y_predict_gridSearch=gridSearch_result.predict(X_test)
print(gridSearch_result.score(X_test,y_test))
print(metrics.confusion_matrix(y_test,y_predict_gridSearch))

In [219]:
plt.plot(r,p,label="logistic")
plt.plot([0,1],[0,1],linestyle="--",label="no_skill")
plt.legend()
plt.xlabel("recall")
plt.ylabel("precision")

In [225]:
np.where(r==(max(p)))

In [237]:
#pd.DataFrame({"r":r,"p":p})
pd.DataFrame(zip(r,p),columns=["r","p"])

In [235]:
list(zip(r,p))